In [2]:
pip install jieba

Note: you may need to restart the kernel to use updated packages.


In [1]:
import codecs
import jieba as jb
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pandas as pd
import nltk 
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perfrom cosine similarity
from nltk import word_tokenize # to create tokens
from nltk.corpus import stopwords # for stop words

In [3]:
from nltk.corpus import stopwords

In [4]:
#中文分词
def SentsTokenizer4Ch(text):
    #sentences = re.split('(。|！|\!|\.|？|\?)',text)
#     sentences = re.split('(。|！|\!|？|\?)',text)
    sentences = re.split('(。|！|\!|？|\?)',text)
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        sent = sent.replace('\r\n','')
        sent = sent.strip()
        new_sents.append(sent)
    return new_sents

#删除所有符号,只保留字母、数字和中文
def remove_punctuation(line):
    #line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
# #中文停用词
# def stopwordslist(filepath):  
#     stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
#     return stopwords  
 
# # #加载停用词
# stopwords = stopwordslist("/Users/baihuiqiu/Desktop/111/EnglishStop Words.txt")


In [5]:
# myfile = "/Users/baihuiqiu/Desktop/111/University of Leeds Education Services.txt"
myfile = "/Users/baihuiqiu/Desktop/111/home.txt"
# myfile = "/Users/baihuiqiu/Desktop/111/teaching_enhancement_scheme.html"
text = codecs.open(myfile, "r", "utf-8").read()
sent_tokens = SentsTokenizer4Ch(text)
sent_tokens[:10]

["Logo University of Leeds Home \nTaught and research student operations \nCommunicating with students: \nTHE THEME OF CONTENT\n\xa0Academic life - this relates to the student journey in line with the academic year.\n\xa0Health and wellbeing - the health and support services/facilities available to students.\n\xa0Study support - the wide range of study related support available to students.\n\xa0Opportunities - what's available to students to help them make the most of their time at Leeds.\n\xa0Life at Leeds - this is about campus, making the most of university life and events that are happening on campus.\nCHANNELS\nThe student communications team manage a number of channels that are used to inform students of university-wide messages, these are:\xa0\nFortnightly email newsletters to undergraduate and taught postgraduate students - previous newsletters are available on the\xa0news section of the student website\nA\xa0current student Facebook group\xa0- this is linked to the official\x

In [6]:
# sent_tokens = text.replace(u'\xa0', ' ')
# # # print(sent_tokens)

In [7]:
# #删除除字母,数字，汉字以外的所有符号
df = pd.DataFrame(sent_tokens, columns=['sent'])
# df['clean_set']=  df['sent'].apply(remove_punctuation)
# df['clean_set'] = df['sent'].rename(columns=lambda x: x.replace("'","").replace('"','')).replace(" ","")
df['clean_set'] = df.rename(columns=lambda x: x.replace("'","").replace('"','')).replace(" ","").replace("©","")

# #分词，并过滤停用词
# # df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jb.cut(x))]))
df.head()


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/r1/33gjkcxn5x73kc_5y6ghkykr0000gn/T/jieba.cache
Loading model cost 0.684 seconds.
Prefix dict has been built successfully.


,sent,clean_set,cut_sent
0,Logo University of Leeds Home \nTaught and res...,Logo University of Leeds Home \nTaught and res...,Logo University of Leeds Home \n Tau...
1,Level refers to the level of a module and its ...,Level refers to the level of a module and its ...,Level refers to the level of a m...
2,Year (called Class in the Classification Tool)...,Year (called Class in the Classification Tool)...,Year ( called Class in the Classific...
3,Undergraduate full time students must study 12...,Undergraduate full time students must study 12...,Undergraduate full time students must ...
4,Undergraduate students who have a start date p...,Undergraduate students who have a start date p...,Undergraduate students who have a st...


首先执行第一步，文本规范化处理。我们将所有的数据转化为小写，删除特殊符号，提取词干。这部分代码我们使用函数step1进行封装：

In [8]:
# function that converts text into lower case and removes special characters

def step1(x):
    for i in x:
        a=str(i).lower()
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

In [9]:
step1(df['cut_sent'])

logo   university   of   leeds   home     taught   and   research   student   operations     communicating   with   students       the   theme   of   content     academic   life       this   relates   to   the   student   journey   in   line   with   the   academic   year       health   and   wellbeing       the   health   and   support   services   facilities   available   to   students       study   support       the   wide   range   of   study   related   support   available   to   students       opportunities       what   s   available   to   students   to   help   them   make   the   most   of   their   time   at   leeds       life   at   leeds       this   is   about   campus     making   the   most   of   university   life   and   events   that   are   happening   on   campus     channels   the   student   communications   team   manage   a   number   of   channels   that   are   used   to   inform   students   of   university   wide   messages     these   are       fortnightly 

可以看到文本已经干净多了。分词指的是将文本字符串切分为词条

In [10]:
 # word tokenizing
    
s='WHAT DOES THE TERM “LEVEL” MEAN?'
words=word_tokenize(s)
print(words)

['WHAT', 'DOES', 'THE', 'TERM', '“', 'LEVEL', '”', 'MEAN', '?']


In [11]:
lemma = wordnet.WordNetLemmatizer() # intializing lemmatizer
lemma.lemmatize('absorbed', pos = 'v')

'absorb'

pos_tag函数返回每个词条在文本中的作用：

In [12]:
pos_tag(nltk.word_tokenize(s),tagset = None) # returns the parts of speech of every word

[('WHAT', 'WP'),
 ('DOES', 'VBZ'),
 ('THE', 'DT'),
 ('TERM', 'NNP'),
 ('“', 'NNP'),
 ('LEVEL', 'NNP'),
 ('”', 'NNP'),
 ('MEAN', 'NNP'),
 ('?', '.')]

我们现在将创建一个函数来整合上面这些环节的代码：



In [13]:
# function that performs text normalization steps

def text_normalization(text):
    text=str(text).lower() # text to lower case
    spl_char_text=re.sub(r'[^ a-z]','',text) # removing special characters
    tokens=nltk.word_tokenize(spl_char_text) # word tokenizing
    lema=wordnet.WordNetLemmatizer() # intializing lemmatization
    tags_list=pos_tag(tokens,tagset=None) # parts of speech
    lema_words=[]   # empty list 
    for token,pos_token in tags_list:
        if pos_token.startswith('V'):  # Verb
            pos_val='v'
        elif pos_token.startswith('J'): # Adjective
            pos_val='a'
        elif pos_token.startswith('R'): # Adverb
            pos_val='r'
        else:
            pos_val='n' # Noun
        lema_token=lema.lemmatize(token,pos_val) # performing lemmatization
        lema_words.append(lema_token) # appending the lemmatized token into a list
    
    return " ".join(lema_words) # returns the lemmatized tokens as a sentence

让我们用这个函数处理数据集：

In [14]:
text_normalization('telling you some stuff about me')

'tell you some stuff about me'

In [15]:
df['lemmatized_text']=df['cut_sent'].apply(text_normalization) # applying the fuction to the dataset to get clean text
df.tail(15)



,sent,clean_set,cut_sent,lemmatized_text
5,"When registering with the University, the stud...","When registering with the University, the stud...","When registering with the University ,...",when register with the university the student ...
6,"If, after the start of any semester's teaching...","If, after the start of any semester's teaching...","If , after the start of any semest...",if after the start of any semester s teach a s...
7,Where a student transfers in credits against a...,Where a student transfers in credits against a...,Where a student transfers in credits...,where a student transfer in credit against a p...
8,There are usually three University assessment ...,There are usually three University assessment ...,There are usually three University a...,there be usually three university assessment p...
9,If a student does not pass enough credits or d...,If a student does not pass enough credits or d...,If a student does not pass enough ...,if a student do not pass enough credit or do n...
10,Any re-sit has to be taken at the next opportu...,Any re-sit has to be taken at the next opportu...,Any re - sit has to be taken at ...,any re sit have to be take at the next opportu...
11,All undergraduate students are entitled to a r...,All undergraduate students are entitled to a r...,All undergraduate students are entitle...,all undergraduate student be entitle to a re s...
12,The student must submit their re-sit applicati...,The student must submit their re-sit applicati...,The student must submit their re - s...,the student must submit their re sit applicati...
13,Re-sit assessment dates\nRe-sit assessments ge...,Re-sit assessment dates\nRe-sit assessments ge...,Re - sit assessment dates \n Re - sit as...,re sit assessment date re sit assessment gener...
14,"If the exam is a “first attempt”, there are no...","If the exam is a “first attempt”, there are no...",If the exam is a “ first attempt ”...,if the exam be a first attempt there be no fee...


下一步是单词嵌入表示，两次词义相近的词，其嵌入表示的距离也相近。有两种模型可以用于这个处理环节：词袋模型和tf-idf模型。

词袋模型:词袋模型是描述文本中出现的单词的一种表示方法。例如，假设我们的词典中包含单词{Playing, is, love}，我们希望矢量化文本“Playing football is love”，那么得到的矢量就是： (1, 0, 1, 1)。

In [16]:
# all the stop words we have 

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
cv = CountVectorizer() # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [18]:
# returns all the unique word from data 

features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,able,about,above,abroad,absent,ac,academic,accept,access,accessible,...,with,withdraw,within,without,work,www,year,yes,you,your
0,0,2,0,0,0,2,2,0,0,1,...,2,0,0,0,0,0,1,0,3,4
1,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,2,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,8,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,3,0,0,0,...,0,0,0,0,0,0,3,0,0,0


In [19]:
# all the stop words we have 

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
Question ='WHAT DOES THE TERM “LEVEL”?' # considering an example query

In [21]:
# checking for stop words

Q=[]
a=Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b=" ".join(Q) 

In [22]:
Question_lemma = text_normalization(b) # applying the function that we created for text normalizing
Question_bow = cv.transform([Question_lemma]).toarray() # applying bow

In [23]:
text_normalization

<function __main__.text_normalization(text)>

In [24]:
Question_bow

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

余弦相似性:余弦相似性是衡量两个矢量相似性的一种指标。其计算方法是用两个矢量的点积除以两个矢量的模的乘积：Cosine Similarity (a, b) = Dot product(a, b) / ||a|| * ||b||, 计算代码如下：

In [25]:
# cosine similarity for the above question we considered.

cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.29538623],
       [0.388057  ],
       [0.12659242],
       [0.17626106],
       [0.30705979],
       [0.1490712 ],
       [0.20675612],
       [0.13151919],
       [0.1860521 ],
       [0.23972644],
       [0.22627417],
       [0.24902912],
       [0.23809164],
       [0.2262621 ],
       [0.20223698],
       [0.30292266],
       [0.        ],
       [0.24194335],
       [0.22733714],
       [0.19227214]])

In [26]:
df['similarity_bow']=cosine_value # creating a new column 

In [27]:
df_simi = pd.DataFrame(df, columns=['cut_sent','similarity_bow']) # taking similarity value of responses for the question we took
df_simi

,cut_sent,similarity_bow
0,Logo University of Leeds Home \n Tau...,0.295386
1,Level refers to the level of a m...,0.388057
2,Year ( called Class in the Classific...,0.126592
3,Undergraduate full time students must ...,0.176261
4,Undergraduate students who have a st...,0.307060
5,"When registering with the University ,...",0.149071
6,"If , after the start of any semest...",0.206756
7,Where a student transfers in credits...,0.131519
8,There are usually three University a...,0.186052
9,If a student does not pass enough ...,0.239726


In [28]:
df_simi_sort = df_simi.sort_values(by='similarity_bow', ascending=False) # sorting the values
df_simi_sort.head()

,cut_sent,similarity_bow
1,Level refers to the level of a m...,0.388057
4,Undergraduate students who have a st...,0.307060
15,Students may be eligible to sit th...,0.302923
0,Logo University of Leeds Home \n Tau...,0.295386
11,All undergraduate students are entitle...,0.249029


In [29]:
threshold = 0.2 # considering the value of p=smiliarity to be greater than 0.2
df_threshold = df_simi_sort[df_simi_sort['similarity_bow'] > threshold] 
df_threshold

,cut_sent,similarity_bow
1,Level refers to the level of a m...,0.388057
4,Undergraduate students who have a st...,0.307060
15,Students may be eligible to sit th...,0.302923
0,Logo University of Leeds Home \n Tau...,0.295386
11,All undergraduate students are entitle...,0.249029
17,Students use the Minerva to access ...,0.241943
9,If a student does not pass enough ...,0.239726
12,The student must submit their re - s...,0.238092
18,Supernumerary modules are modules chos...,0.227337
10,Any re - sit has to be taken at ...,0.226274


In [30]:
index_value = cosine_value.argmax() # returns the index number of highest value
index_value 

1

让我们输出其查询文本看看是否相关：

In [31]:
(Question)

'WHAT DOES THE TERM “LEVEL”?'

In [32]:
# sent_tokens = text.replace(u'\xa0', ' ')
df['cut_sent'].loc[index_value].replace(u'\xa0', ' ') # The text at the above index becomes the response for the question

'Level   refers   to   the   level   of   a   module   and   its   credits . \n For   example : \n CHEM1002   -   a   level   1   module   with   a   credit   value   of   20       i . e .   20   level - 1   credits \n BIOC2667   -   a   level   2   module   with   a   credit   value   of   10 \n CIVE3772   -   a   level   3   module   with   a   credit   value   of   40     i . e .   40   level - 3   credits \n LUBS5778M   -   a   level   M   module   with   a   credit   value   of   30 \n FLTU1559   -   a   level   1   module   with   a   credit   value   of   10   which   can   be   taken   in   any   year   of   study   that   the   programme   specification   allows . \n This   is   also   explained   in   the   Rules   for   Award . \n \n WHAT   DOES   THE   TERM   “ YEAR ”   MEAN ?'

TF-IDF模型:tf指的是词频，表示单词在当前文档中出现的频率，idf指的是逆文档频率，表示单词在文档集中出现的频率倒数。这里我们说的文档表示一个样本，文档集表示所有的样本。

In [33]:
Question1 ='WHAT DOES THE TERM “YEAR” MEAN?'
Question2 ='HOW MUCH ARE THE RE-SIT FEES?'

In [34]:
# using tf-idf

tfidf=TfidfVectorizer() # intializing tf-id 
x_tfidf=tfidf.fit_transform(df['lemmatized_text']).toarray() # transforming the data into array

In [35]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray() # applying tf-idf


In [36]:
# returns all the unique word from data with a score of that word

df_tfidf=pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names()) 
df_tfidf.head()


,able,about,above,abroad,absent,ac,academic,accept,access,accessible,...,with,withdraw,within,without,work,www,year,yes,you,your
0,0.0,0.066866,0.0,0.0,0.000000,0.066866,0.041930,0.0,0.0,0.038035,...,0.039539,0.0,0.0,0.0,0.0,0.0,0.018688,0.0,0.1003,0.133733
1,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.230289,0.0,0.0,0.0,0.0,0.0,0.087075,0.0,0.0000,0.000000
2,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.064181,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.457674,0.0,0.0000,0.000000
3,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.092489,0.0,0.0000,0.000000
4,0.0,0.000000,0.0,0.0,0.034163,0.000000,0.071224,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.063487,0.0,0.0000,0.000000


In [37]:
cos=1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')  # applying cosine similarity
cos

array([[0.14688982],
       [0.15682044],
       [0.20387195],
       [0.08118346],
       [0.14537092],
       [0.05269408],
       [0.07315791],
       [0.05190082],
       [0.09042639],
       [0.12971193],
       [0.1152888 ],
       [0.15212475],
       [0.08628485],
       [0.07776947],
       [0.07917328],
       [0.11460062],
       [0.        ],
       [0.10414171],
       [0.17504604],
       [0.06864426]])

In [38]:
df['similarity_tfidf']=cos # creating a new column 
df_simi_tfidf = pd.DataFrame(df, columns=['cut_sent','similarity_tfidf']) # taking similarity value of responses for the question we took
df_simi_tfidf 


,cut_sent,similarity_tfidf
0,Logo University of Leeds Home \n Tau...,0.146890
1,Level refers to the level of a m...,0.156820
2,Year ( called Class in the Classific...,0.203872
3,Undergraduate full time students must ...,0.081183
4,Undergraduate students who have a st...,0.145371
5,"When registering with the University ,...",0.052694
6,"If , after the start of any semest...",0.073158
7,Where a student transfers in credits...,0.051901
8,There are usually three University a...,0.090426
9,If a student does not pass enough ...,0.129712


In [39]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False) # sorting the values
df_simi_tfidf_sort.head(10)

,cut_sent,similarity_tfidf
2,Year ( called Class in the Classific...,0.203872
18,Supernumerary modules are modules chos...,0.175046
1,Level refers to the level of a m...,0.156820
11,All undergraduate students are entitle...,0.152125
0,Logo University of Leeds Home \n Tau...,0.146890
4,Undergraduate students who have a st...,0.145371
9,If a student does not pass enough ...,0.129712
10,Any re - sit has to be taken at ...,0.115289
15,Students may be eligible to sit th...,0.114601
17,Students use the Minerva to access ...,0.104142


In [40]:
threshold = 0.2 # considering the value of p=smiliarity to be greater than 0.2
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold] 
df_threshold

,cut_sent,similarity_tfidf
2,Year ( called Class in the Classific...,0.203872


In [41]:
index_value1 = cos.argmax() # returns the index number of highest value
index_value1

2

In [42]:
Question1

'WHAT DOES THE TERM “YEAR” MEAN?'

In [43]:
df['cut_sent'].loc[index_value1].replace(u'\xa0', ' ').replace('\n', ' ')  # returns the text at that index

'Year   ( called   Class   in   the   Classification   Tool )   is   the   study   ( Academic )   year   that   a   student   studies .     Programmes   are   made   up   of   modules   and   credits   to   a   predetermined   standard   and   are   designed   to   run   over   a   number   of   study   years .   For   example :   First   year   of   study   on   BA   English   A   second   year   student   on   the   BSc   Math   programme     Penultimate   year   of   study   on   BEng   Final   year   of   study   on   BA   Education   ( part - time )   –   which   could   be   the   student ’ s   6th   year   of   study .   This   is   also   explained   in   the   Rules   for   Award .     HOW   MANY   CREDITS   CAN   A   STUDENT   STUDY ?'

In [44]:
# Function that removes stop words and process the text

def stopword_(text):   
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
    stop=stopwords.words('english')
    lema=wordnet.WordNetLemmatizer()
    lema_word=[]
    for token,pos_token in tag_list:
        if token in stop:
            continue
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)
        lema_word.append(lema_token)
    return " ".join(lema_word) 

In [45]:
# defining a function that returns response to query using bow

def chat_bow(text):
    s=stopword_(text)
    lemma=text_normalization(s) # calling the function to perform text normalization
    bow=cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value=cosine_value.argmax() # getting index value 
    return df['cut_sent'].loc[index_value]

# TF_IDF model

In [46]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma=text_normalization(text) # calling the function to perform text normalization
    tf=tfidf.transform([lemma]).toarray() # applying tf-idf
    cos=1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value=cos.argmax() # getting index value 
    return df['cut_sent'].loc[index_value]

In [47]:
chat_tfidf('Hi, I want to know SUPERNUMERARY MODULES, Can you please tell me?').replace('\n', '').replace('\xa0', '')

'Supernumerary   modules   are   modules   chosen   by   students   for   interest   only   and   are   taken   as   extra   credits   on   top   of   the   normal   amount   of   credits   for   the   year   ( i . e .   over   120   credits   in   any   undergraduate   Programme   Year   or   over   180   in   any   postgraduate   Programme   Year ) .    This   means   the   credits   gained   in   supernumerary   modules   cannot   be   counted   for   progression   ( e . g .   a   student   passing   70   credits   plus   a   10 - credit   supernumerary   module   is   not   permitted   to   include   the   supernumerary   credits   to   give   them   the   80   credits   required   to   progress )   or   classification   purposes .  Where   any   supernumerary   modules   ( i . e .   over   120   credits   in   any   undergraduate   Programme   Year   or   over   180   in   any   postgraduate   Programme   Year )   are   required   to   meet   the   learning   outcomes   of   a   s

In [48]:
chat_tfidf('Excuse me, I would like to know about AD-HOC/AD PERSONAM PROGRAMMES').replace('\n', '').replace('\xa0', '')

'In   exceptional   circumstances   it   may   be   necessary   to   make   time - limited   amendments   to   a   programme   of   study . \u2028 Where   these   circumstances   relate   to   one   student   or   a   small   number   of   individual   students ,   the   amended   programme   is   referred   to   as   an   ad   personam   programme   variation .  Where   the   circumstances   relate   to   an   entire   cohort   of   students ,   it   is   an   ad   hoc   programme   variation .  \u2028 Any   ad   personam   or   ad   hoc   programme   variation   is   subject   to   the   approval   of   the   relevant   Pro   Dean   for   Student   Education ,   acting   on   behalf   of   the   Faculty   Taught   Student   Education   Committee   ( FTSEC ) .  The   Head   of   School   must   make   a   case   for   an   ad   personam   or   an   ad   hoc   programme   and   forward   this   onto   the   Exams   and   Progress   team ,   who   will   seek   approval   from   the   P

In [49]:
chat_tfidf('WHAT DOES THE TERM “YEAR” MEAN?').replace('\n', '').replace('\xa0', '')

'Year   ( called   Class   in   the   Classification   Tool )   is   the   study   ( Academic )   year   that   a   student   studies .   Programmes   are   made   up   of   modules   and   credits   to   a   predetermined   standard   and   are   designed   to   run   over   a   number   of   study   years .  For   example :  First   year   of   study   on   BA   English  A   second   year   student   on   the   BSc   Math   programme   Penultimate   year   of   study   on   BEng  Final   year   of   study   on   BA   Education   ( part - time )   –   which   could   be   the   student ’ s   6th   year   of   study .  This   is   also   explained   in   the  Rules   for   Award .   HOW   MANY   CREDITS   CAN   A   STUDENT   STUDY ?'

In [50]:
chat_tfidf('CAN A STUDENT RETAKE A MODULE THEY HAVE ALREADY PASSED?').replace('\n', '').replace('\xa0', '')

'All   undergraduate   students   are   entitled   to   a   re - sit   in   August   ( unless   specifically   precluded   by   the   school ) .  \u2028 Following   the   August   re - assessment   period ,  if   an   Undergraduate   student   with   a   start   date   prior   to   academic   term   2016 / 2017 ,   has   failed   a   module   which   is   a   must   pass   to   progress   module ,   or   the   student   has   not   achieved   sufficient   credits   to   progress ,   then   the   student   must   become   an   external   student   the   following   session .   If   the   Undergraduate   student   has   a   start   date   in   academic   term   2016 / 2017  they   are   only   permitted   two   attempts   to   pass   a   module .    If   the   student   has   failed   a   module   which   is   a   must   pass   to   progress   module ,   or   the   student   has   not   achieved   sufficient   credits   to   progress ,   having   been   unsuccessful   in   their   final 

In [51]:
chat_tfidf('IF A STUDENT TRANSFERS TO A DIFFERENT PROGRAMME OF STUDY, DO THEY HAVE TO STUDY 120 CREDITS IF THEY HAVE ALREADY ACHIEVED SOME OF THE RELEVANT CREDITS?').replace('\n', '').replace('\xa0', '')

'Where   a   student   transfers   in   credits   against   a   programme   –   they   are   exempted   from   those   credits   and   do   not   have   to   pick   up   a   full   diet .  For   example :  Masters   programme   of   180   credits ,   students   can   transfer   in   up   to   60   credits ,   and   just   study   the   remaining   120   credits .    They   do   not   need   to   top   up   to   180   credits   with   other   modules .    Refer   to  Transfer   to   a   new   programme  for   more   information .      ASSESSMENT   RELATED   QUESTIONS   INCLUDING   RESITS ,   RESIT   FEES ,   EXTERNAL   AND   OVERSEAS   ASSESSMENTS  WHEN   ARE   THE   UNIVERSITY   ASSESSMENT   PERIODS ?'

In [52]:
chat_tfidf('HOW MUCH ARE THE RE-SIT FEES?').replace('\n', '').replace('\xa0', '')

'If   the   exam   is   a   “ first   attempt ” ,   there   are   no   fees .  If   the   exam   is   not   a   “ first   attempt ” ,   the  re - sit   fees  are : \u2028 £ 2.00   per   credit   ( i . e .   £ 20.00   for   a   10   credit   module ) \u2028 plus \u2028 £ 50.00   administration   fee   ( increases   to   £ 100   after   the   deadline ) .   This   is   a   one - off   fee ,   which   is   the   same   regardless   of   how   many   modules   the   student   takes .  Fees   for   re - sits   taken   in   August   will   be   charged   to   the   student ’ s   University   account   after   31   July .   Fees   for   re - sits   taken   at   other   times   of   the   year   will   be   charged   after   the   application   has   been   processed .    You   will   receive   a   statement   of   fees   by   email   and   must   pay   immediately .    Failure   to   pay   will   result   in   late   payment   administration   charges   of   £ 20   per   month   until   the  

In [53]:
chat_tfidf('HOW DO I FIND MY STAFF ONLINE TIMETABLE?').replace('\n', '').replace('\xa0', '')

"Re - sit   assessment   dates  Re - sit   assessments   generally   take   place   in   August   ( during   the   two   weeks   prior   to   the   August   bank   holiday ) .   ( See  Assessment   Timetables  for   more   information   on   cohorts ) .  Unfortunately   it   isn ' t   possible   to   find   out   the   exact   date   of   an   assessment   before   the   students   apply   to   re - sit .   This   is   because   the   timetable   is   produced   once   the   information   is   known   of   which   modules   students   have   applied   for .  Students   should   view   their   personal   assessment   timetable   via  Minerva  ( see  Assessment   Timetable   publication   dates ) .  Students   should   check   that   they   have   been   entered   for   the   appropriate   exams ,   and   tell   the   Assessment   &   Progress   Team   immediately   about   any   discrepancies .   Please   refer   to  Assessment   periods   and   timetable   publication   dates  for   fu

In [54]:
chat_tfidf('VIEW YOUR CLASSLISTS').replace('\n', '').replace('\xa0', '')

'When   registering   with   the   University ,   the   student   will   automatically   be   registered   for   all   of   their   compulsory   examinations / assessments   for   the   programme   of   study   they   have   chosen .   Refer   to  When   to   enrol   on   your   modules  on   the   For   Students   site   for   further   information   on   modules   and   how   and   when   to   enrol   on   to   modules .  WHEN   CAN   A   STUDENT   CHANGE   A   MODULE ?'

In [55]:
chat_tfidf('HOW MANY ATTEMPTS DOES A STUDENT HAVE TO PASS A MODULE?').replace('\n', '').replace('\xa0', '')

'All   undergraduate   students   are   entitled   to   a   re - sit   in   August   ( unless   specifically   precluded   by   the   school ) .  \u2028 Following   the   August   re - assessment   period ,  if   an   Undergraduate   student   with   a   start   date   prior   to   academic   term   2016 / 2017 ,   has   failed   a   module   which   is   a   must   pass   to   progress   module ,   or   the   student   has   not   achieved   sufficient   credits   to   progress ,   then   the   student   must   become   an   external   student   the   following   session .   If   the   Undergraduate   student   has   a   start   date   in   academic   term   2016 / 2017  they   are   only   permitted   two   attempts   to   pass   a   module .    If   the   student   has   failed   a   module   which   is   a   must   pass   to   progress   module ,   or   the   student   has   not   achieved   sufficient   credits   to   progress ,   having   been   unsuccessful   in   their   final 

In [56]:
chat_tfidf('Hello, I want to Know how many credits can I study?').replace('\n', '').replace('\xa0', '')

"Undergraduate   full   time   students   must   study   120   credits   per   year   on   a   full - time   programme .  Taught   Postgraduate   full   time   students   must   study   a   minimum   of   180   credits ,   of   which   at   least   135   must   be   ‘ M '   level   credits .  Refer   to  Modules   and   programmes   explained ,  the  Rules   for   Award  and / or   the  Ordinances  for   more   information .  HOW   MANY   ATTEMPTS   DOES   A   STUDENT   HAVE   TO   PASS   A   MODULE ?"

In [57]:
chat_tfidf('HOW MUCH ARE THE RE-SIT FEES?').replace('\n', '').replace('\xa0', '').replace('\u2028', '')


'If   the   exam   is   a   “ first   attempt ” ,   there   are   no   fees .  If   the   exam   is   not   a   “ first   attempt ” ,   the  re - sit   fees  are :  £ 2.00   per   credit   ( i . e .   £ 20.00   for   a   10   credit   module )  plus  £ 50.00   administration   fee   ( increases   to   £ 100   after   the   deadline ) .   This   is   a   one - off   fee ,   which   is   the   same   regardless   of   how   many   modules   the   student   takes .  Fees   for   re - sits   taken   in   August   will   be   charged   to   the   student ’ s   University   account   after   31   July .   Fees   for   re - sits   taken   at   other   times   of   the   year   will   be   charged   after   the   application   has   been   processed .    You   will   receive   a   statement   of   fees   by   email   and   must   pay   immediately .    Failure   to   pay   will   result   in   late   payment   administration   charges   of   £ 20   per   month   until   the   debt   is   repai

In [58]:
chat_tfidf('WHAT DOES THE TERM “LEVEL” MEAN?').replace('\n', '').replace('\xa0', '')

'Level   refers   to   the   level   of   a   module   and   its   credits .  For   example :  CHEM1002   -   a   level   1   module   with   a   credit   value   of   20     i . e .   20   level - 1   credits  BIOC2667   -   a   level   2   module   with   a   credit   value   of   10  CIVE3772   -   a   level   3   module   with   a   credit   value   of   40    i . e .   40   level - 3   credits  LUBS5778M   -   a   level   M   module   with   a   credit   value   of   30  FLTU1559   -   a   level   1   module   with   a   credit   value   of   10   which   can   be   taken   in   any   year   of   study   that   the   programme   specification   allows .  This   is   also   explained   in   the  Rules   for   Award .   WHAT   DOES   THE   TERM   “ YEAR ”   MEAN ?'

In [59]:
GREETING_INPUTS = ("hello", "hi", "how are you", "Excuse me", "are you here?",'yeah',"Thanks")
GREETING_RESPONSES = ["hello", "Excuse me, How can I help you?", "yes,I'm here", "I'm fine","Hello, nice to meet you！"]

def greeting(sentence):
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    text = rule.sub('',sentence)
    if text in GREETING_INPUTS:
        return random.choice(GREETING_RESPONSES)
    wordlist =  [w for w in jb.cut(sentence)]
    for word in wordlist:
        if word in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
        
greeting('are you here?')

In [60]:
greeting('有人在吗？')

In [61]:
def response(user_response):
    robo_response=''
    user_response  = remove_punctuation(user_response)
    user_response= " ".join([w for w in list(jb.cut(user_response)) if w not in stop])
    cut_sent = df.cut_sent.values.tolist()
    cut_sent.append(user_response)
    tfidf = TfidfVectorizer()

    tfidf_vec = tfidf.fit_transform(cut_sent)

    cos_sim = cosine_similarity(tfidf_vec[-1], tfidf_vec)
    idx=cos_sim.argsort()[0][-2]
    flat = cos_sim.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if(req_tfidf==0):
        robo_response=robo_response+"Sorry, I don't quite understand what you mean, I'm still a rookie...!"
        return(robo_response)
    else:
        robo_response = robo_response+df.sent.values[idx]
        return(robo_response) 
        



In [ ]:
flag=True
print("Chatbot: My name is Carol. I can answer your questions about school education information. If you wish to exit, please enter:bye !")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='Thanks' or user_response=='Thank you' ):
            flag=True
            print("Chatbot: No worries！")
        else:
            if(greeting(user_response)!=None):
                print("Chatbot: "+greeting(user_response))
            else:
                print("Chatbot: ",end="")
                print(response(user_response))
               
    else:
        flag=False
        print("Carol: Bye! Feel free to ask again!")    

Chatbot: My name is Carol. I can answer your questions about school education information. If you wish to exit, please enter:bye !
